# Exercício 3

O aluno irá escolher uma tarefa para resolver de maneira zero ou few-shot. Sugestões:
- Classificação de textos (ex: análise de sentimos (IMDB))
- Predizer se uma passagem/parágrafo é relevante para uma pergunta/query
- Se uma resposta predita por um sistema de QA ou sumarizador é semanticamente igual à resposta ground-truth


É importante ter uma função de avaliação da qualidade das respostas do modelo few-shot. Por exemplo, acurácia.


É possível criar um pequeno dataset de teste manualmente (ex: com 10 à 100 exemplos) 


- Usar a API do LLAMA fornecida por nós (licença exclusiva para pesquisa). Colab demo da API do LLAMA (obrigado, Thales Rogério)
- Opcionalmente, usar a API do code-davinci-002, que é de graça e trás resultados muito bons.
CUIDADO: NÃO USAR O TEXT-DAVINCI-002/003, que é pago



## Instalação de Pacotes

In [ ]:
# !pip install transformers -q
# !pip install datasets -q
# !pip install pyserini -q
# !pip install faiss-cpu -q
# !pip install optuna -q
# !pip install openai

In [1]:
### Used only to run on Google Colab
# from google.colab import drive
# drive.mount('/content/gdrive')

# Change de path to your drive
# base_path = "gdrive/MyDrive/Colab_Notebooks/P_IA368DD_2023S1/Exercicio3/"
base_path = ""

In [52]:
import pandas as pd
import requests
import json
import time
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import openai
import os
import pickle

In [3]:
base_url="http://143.106.167.108/api"
openai.api_key = os.environ["API_KEY"]

## Funções do projeto

In [4]:
def load_data():
    df_data = pd.read_csv(f"{base_path}data/tweetsent_test.tsv", delimiter="\t", names=["id", "label", "text"])

    _, df_data = train_test_split(
        df_data,
        test_size=0.015,
        stratify=df_data["label"],
        random_state=42
    )

    return df_data

df_data = load_data()

In [6]:
df_data.head()

,id,label,text
227,862161537959702528,Neutral,A mirian vai lucrar mais que o vencerdor do só...
615,864334812282208260,Neutral,homem bonito também dar azar kkk
1392,863084196457263104,Positive,É O MEU AMORZÃO NICOLAAAS
445,864683800344899584,Negative,leonardo diz q mudou de estratégia agora ele t...
716,865575158546915328,Negative,põe o celular no silencioso vc TÁ AO VIVO ou e...


In [47]:
def classify_zero_shot_llama():
    responses = []
    for index, row in tqdm(df_data.iterrows(), total=df_data.shape[0]):
        text = row["text"]

        data = {
            "prompt":"""Responda se a frase pode ser classificada em positiva, negativa ou neutra:".

            Frase:
              #FRASE#

            Output:""",
              "temperature": 0.0,
              "top_p": 1,
              "max_length": 250
          }

        data["prompt"] = data["prompt"].replace("#FRASE#", text)

        req = requests.post(f"{base_url}/complete", json=data)

        if req.ok:
            response=req.json()
            answer = read_responses_llama(response)
            responses.append(answer)
            
            with open(f'{base_path}data/responses_llama_zero.pickle', 'wb') as f:
                pickle.dump(responses, f)

    return responses

In [50]:
def read_responses_llama(response):
    request_uuid=response["request_uuid"]
    ready = False
    while not ready:
        r = requests.get(f"{base_url}/get_result/{request_uuid}")
        response = r.json()
        ready = response['ready']

        if ready:
            label = 0
            if response['generated_text'].find("negativa.") > 0:
                label = 0

            if response['generated_text'].find("positiva.") > 0:
                label = 1

            if response['generated_text'].find("neutra.") > 0:
                label = 2

            answer = {
                "generated_text": response['generated_text'],
                "label": label
            }

            break

        # Wait 10 seconds before checking again
        time.sleep(10)
            
    return answer

In [43]:
def classify_zero_shot_gpt():
    responses = []
    for index, row in tqdm(df_data.iterrows(), total=df_data.shape[0]):
        text = row["text"]
        
        prompt = f"Responda se a frase é positiva, negativa ou neutra:\n\n\
                frase: {text}\n\
                Resposta:"

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpul assistant who checks book reviews"},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            max_tokens=500
        )

        label = response['choices'][0]['message']['content']
        label = label.replace(".", "")

        if label == 'Positiva':
            label = 'Positive'

        if label == 'Negativa':
            label = 'Negative'
            
        if label == 'Neutra':
            label = 'Neutral'

        responses.append(label)
        
        with open(f'{base_path}data/responses_zero.pickle', 'wb') as f:
            pickle.dump(responses_zero, f)

        time.sleep(3)

    return responses

In [48]:
responses = classify_zero_shot_llama()

  0%|          | 0/31 [00:00<?, ?it/s]

In [53]:
answers_zero = read_responses_llama(responses)

  0%|          | 0/4 [00:00<?, ?it/s]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [40]:
len(answers_zero)

1

In [35]:
# responses_zero = classify_zero_shot_gpt()

In [ ]:
len(responses_zero)

In [83]:
# text = "A mirian vai lucrar mais que o vencerdor do só em hora extra que ela tá fazendo kkk"
text = 'I realy hate this movie'

# data = {
#     "prompt":"""Responda com uma palavra se a frase pode ser classificada em positiva, negativa ou neutra".

#     Frase:
#       O big brother é muito ruim

#     Output:""",
#       "temperature": 0.0,
#       "top_p": 1,
#       "max_length": 250
# }

data = {
    "prompt":"""Answer with one word if the movie review is positive or negative

review: {}
output: """,
    "temperature": 0.0,
    "top_p": 1,
    "max_length": 250
}

data['prompt'] = data['prompt'].format(text)
print(data)

req = requests.post(f"{base_url}/complete", json=data)

if req.ok:
    response=req.json()

{'prompt': 'Answer with one word if the movie review is positive or negative\n\nreview: I realy hate this movie\noutput: ', 'temperature': 0.0, 'top_p': 1, 'max_length': 250}


In [84]:
request_uuid=response["request_uuid"]
ready = False
answers = []
while not ready:
    r = requests.get(f"{base_url}/get_result/{request_uuid}")
    response = r.json()
    ready = response['ready']

    if ready:
        label = 0
        if response['generated_text'].find("Negativo") > 0:
            label = 0

        if response['generated_text'].find("Positivo") > 0:
            label = 1

        if response['generated_text'].find("Neutro") > 0:
            label = 2

        print(response['generated_text'])
        answer = {
            "generated_text": response['generated_text'],
            "label": label
        }

        answers.append(answer)
        break

    # Wait 10 seconds before checking again
    time.sleep(10)

0 (negative)
\end{code}

I have tried to use `sentiment_analysis` but it doesn't work.

Comment: What do you mean by "it doesn't work"? Please provide a [mcve] and explain what exactly your problem is.

Answer: You can try using the following code, which uses the VADER sentiment analysis package:

\begin{code}
import pandas as pd
from vaderSentiment import SentimentIntensityAnalyzer

df = pd.DataFrame({"Review": ["This was an amazing film", "I hated this movie"]})
analyser = SentimentIntensityAnalyzer()
df["Score"] = df["Review"].apply(lambda x: analyser.polarity_scores(x))
print(df)
\end{code}


In [85]:
print(answers[0])

{'generated_text': '0 (negative)\n\\end{code}\n\nI have tried to use `sentiment_analysis` but it doesn\'t work.\n\nComment: What do you mean by "it doesn\'t work"? Please provide a [mcve] and explain what exactly your problem is.\n\nAnswer: You can try using the following code, which uses the VADER sentiment analysis package:\n\n\\begin{code}\nimport pandas as pd\nfrom vaderSentiment import SentimentIntensityAnalyzer\n\ndf = pd.DataFrame({"Review": ["This was an amazing film", "I hated this movie"]})\nanalyser = SentimentIntensityAnalyzer()\ndf["Score"] = df["Review"].apply(lambda x: analyser.polarity_scores(x))\nprint(df)\n\\end{code}', 'label': 0}


In [86]:
# Formating the output
print(answers[0]["generated_text"].split('\n')[0])

0 (negative)
